# Dekoratory

In [37]:
import functools

def shouter(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('Before call ---- ', func.__name__)
        result = func(*args, **kwargs)
        print('After call ---- ', func.__name__)
        return result
    
    wrapper.__name__ = wrapper.__name__ + "_decorated"
    return wrapper

In [40]:
@shouter
def foo():
    """foo functions"""
    print('foo()')
    return "foo_42"

In [41]:
foo.__name__

'foo_decorated'

In [33]:
foo.__doc__

'foo functions'

In [19]:
foo()

Before call ----  wrapper
Before call ----  foo
foo()
After call ----  foo
After call ----  wrapper


'foo_42'

In [20]:
foo = shouter(foo)

In [23]:
from typing import Callable
from typing import Any


def disable(func: Callable[..., Any]) -> Callable[..., Any | None]:
    @functools.wraps(func)
    def _disable(*args, **kwargs):
        pass
    return _disable

@shouter
def bar(n):
    print(f"bar({n})")

In [24]:
bar(13)

# Dekoratory z parametrami

In [44]:
def tag(tagname):
    def tag_decorator(func):
        @functools.wraps(func)
        def _tag_decorator(*args, **kwargs):
            tag_before = f"<{tagname}>"
            tag_after = f"</{tagname}>"
            return tag_before + func(*args, **kwargs) + tag_after
        return _tag_decorator
    return tag_decorator        

@tag("h1")
@tag("b")
def output(data):
    return data

In [45]:
output("Text")

'<h1><b>Text</b></h1>'

# Rejestracja za pomocą dekoratorów

In [64]:
import collections


class EventRegistry:
    def __init__(self) -> None:
        self.registry = collections.defaultdict(list)

    def on(self, *events):
        def _on(function: Callable) -> Callable:
            for event in events:
                self.registry[event].append(function)
            return function
        return _on
    
    def fire(self, event: str, *args, **kwargs):
        for function in self.registry[event]:
            function(*args, **kwargs)

In [79]:
events = EventRegistry()

@events.on('start', 'success')
def start(service_name: str):
    print(f'Starting service {service_name}')

@events.on('error')
def teardown():
    print('Shutting down system')

class Printer:
    def __init__(self, id: int):
        self.id = id
    
    def run(self, service_name: str):
        print(f'Running printer: {self.id}, {service_name}')

printer = Printer(42)

bound_run = functools.partial(Printer.run, printer)

bound_run = events.on('start')(bound_run) # dynamic registration

In [80]:
events.fire('start', "Deamon665")

Starting service Deamon665
Running printer: 42, Deamon665
